<a href="https://colab.research.google.com/github/Icesuu/FYP/blob/main/VLPNR_under_challenging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!pip install imutils
!pip install roboflow
!pip install pytube --upgrade
!pip install torchinfo
!pip install pillow
!pip install easyocr
#!pip install pytesseract
#!pip install tesseract
!pip install super-resolution
!pip install numpy==1.23
!pip install super-gradients

In [ ]:
from super_gradients.training import models
from super_gradients.training import Trainer
from super_gradients.training import dataloaders
from super_gradients.training.dataloaders.dataloaders import coco_detection_yolo_format_train, coco_detection_yolo_format_val
from IPython.display import clear_output
from super_gradients.training.losses import PPYoloELoss
from super_gradients.training.metrics import DetectionMetrics_050
from super_gradients.training.models.detection_models.pp_yolo_e import PPYoloEPostPredictionCallback
from PIL import Image
from torchinfo import summary
import super_gradients.training.utils as utils
import cv2
import numpy as np
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
from roboflow import Roboflow
import imutils
import easyocr
import time
from super_resolution import cartoon_upsampling_8x
from super_resolution import cartoon_upsampling_4x


In [ ]:
rf = Roboflow(api_key="D9VG7ZKN80WdwYIchywm")
project = rf.workspace("project-p5nyc").project("license-plate-wh9yk")
dataset = project.version(1).download("yolov8")

In [ ]:
!gdown "https://drive.google.com/uc?id=1uGJLdRHa7AI7UiMUKO9FJz95OQ0LKPl1"
#!gdown "https://drive.google.com/drive/folders/uc?id=1-0cC430x5wHPFzfzi37b2MN7uUyktQWE"

In [ ]:
CHECKPOINT_DIR = '/content/checkpoints'
trainer = Trainer (experiment_name = 'apnr_yolonas_run', ckpt_root_dir=CHECKPOINT_DIR)

In [ ]:
dataset_params ={
    'data_dir':'/content/License-Plate-1',
    'train_images_dir':'train/images',
    'train_labels_dir':'train/labels',
    'val_images_dir':'valid/images',
    'val_labels_dir':'valid/labels',
    'test_images_dir':'test/images',
    'test_labels_dir':'test/labels',
    'classes':['license']
}


In [ ]:
train_data = coco_detection_yolo_format_train(
    dataset_params={
        'data_dir': dataset_params['data_dir'],
        'images_dir': dataset_params['train_images_dir'],
        'labels_dir': dataset_params['train_labels_dir'],
        'classes': dataset_params['classes']
    },
    dataloader_params={
        'batch_size':16,
        'num_workers':2
    }
)

val_data = coco_detection_yolo_format_val(
    dataset_params={
        'data_dir': dataset_params['data_dir'],
        'images_dir': dataset_params['val_images_dir'],
        'labels_dir': dataset_params['val_labels_dir'],
        'classes': dataset_params['classes']
    },
    dataloader_params={
        'batch_size':16,
        'num_workers':2
    }
)

test_data = coco_detection_yolo_format_val(
    dataset_params={
        'data_dir': dataset_params['data_dir'],
        'images_dir': dataset_params['test_images_dir'],
        'labels_dir': dataset_params['test_labels_dir'],
        'classes': dataset_params['classes']
    },
    dataloader_params={
        'batch_size':16,
        'num_workers':2
    }
)

clear_output()

In [ ]:
best_model = models.get('yolo_nas_s',
                        num_classes = len(dataset_params['classes']),
                        checkpoint_path = "/content/ckpt_best.pth")

****

In [ ]:
#img_url = '/content/ums.jpg'
#best_model.predict(img_url).show()

# **Run here**

In [ ]:
t1 = time.perf_counter()
t4 = time.perf_counter()
img_path = '/content/11.jpg'
super_img = cartoon_upsampling_4x(img_path,'/content/super.jpg')

In [ ]:
img_url = cv2.imread('/content/super.jpg', cv2.IMREAD_UNCHANGED)


img_detect = cv2.addWeighted(img_url, 0.8, img_url, 0, 0)



# sharpening
kernel = np.array([[0, -1, 0],
                   [-1, 5, -1],
                   [0, -1, 0]])
img_detect = cv2.filter2D(img_detect, -1, kernel)

t5 = time.perf_counter()

#img_detect = cv2.resize(img_detect, (800, 600))

detection = best_model.predict(img_detect)
detection.show()

In [ ]:
bboxes = detection.prediction.bboxes_xyxy
print("bbox: ", bboxes)

In [ ]:
bbox = bboxes[0]


# Convert bounding box coordinates to integers
crop_box = (int(bbox[0]), int(bbox[1]), int(bbox[2]), int(bbox[3]))

# Crop the image
img2 = img_detect[crop_box[1]:crop_box[3], crop_box[0]:crop_box[2]]
img2 = cv2.bitwise_not(img2)

cv2_imshow(img2)


In [ ]:
reader = easyocr.Reader(['en'])

# Adjust contrast and brightness
alpha = 0.8 # Contrast control (1.0 means no change)
beta = 30    # Brightness control
img2 = cv2.convertScaleAbs(img2, alpha=alpha, beta=beta)



cv2_imshow(img2)

In [ ]:
# Perform OCR on the image
results = reader.readtext(img2)

# Display the image with bounding boxes around detected characters
for (bbox, text, prob) in results:
    (top_left, top_right, bottom_right, bottom_left) = bbox
    top_left = tuple(map(int, top_left))
    bottom_right = tuple(map(int, bottom_right))
    cv2.rectangle(img2, top_left, bottom_right, (0, 255, 0), 2)
    cv2.putText(img2, f'{text} ({prob:.2f})', top_left, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

t2 = time.perf_counter()
t3 = t2-t1

In [ ]:

# Display the annotated image

plt.subplot(321)
plt.imshow(img_url)
plt.title('Original')
plt.axis('off')

plt.subplot(323)
plt.imshow(img_detect)
plt.title('Image Pre-processed')
plt.axis('off')

plt.subplot(324)
plt.imshow(cv2.cvtColor(img2, cv2.COLOR_BGR2RGB))
plt.title('Detected License Plate')
plt.axis('off')

plt.subplot(325)
detected_text = [f'{text} ({prob:.2f})' for (bbox, text, prob) in results]
plt.text(0.2, 0.5, '\n'.join(detected_text), fontsize=10,verticalalignment='center')
plt.title('Recognized Character')
plt.axis('off')

plt.subplot(322)
plt.imshow(super_img)
plt.title('Super resolution')
plt.axis('off')

plt.subplot(326)
plt.annotate(t3,(.5,.5),ha='center',va='center')
plt.title("Time Taken(s)")
plt.axis('off')

plt.figsize=10,10
plt.show()



In [ ]:
t6 = t5-t4
plt.subplot(111)
plt.annotate(t6,(.5,.5),ha='center',va='center')
plt.title("Time Taken(s)")
plt.axis('off')



In [ ]:
t7=t3-t6
plt.subplot(111)
plt.annotate(t7,(.5,.5),ha='center',va='center')
plt.title("Time Taken(s)")
plt.axis('off')